In [ ]:
#BASE CODE NO Parallelism or fancy logging
"""
This is the script to get all the option.ticks and populate the dataabse correctly for future use
1. initiates tickers in the options.tickers table and returns unique ticker id called sid
2. constructs all the expiration dates that exist and inserts them into options.expirations by sid
3. retieves available dates of data for every expiration in the sid
4. For every day available populates unique option contract into options.contracts by sid
5. for every contract_id populates a timeseriesdb with 1 minute date 
6. Logs avgg.contrctacts/expiration, avg. ticks/contract, total ticks, total exp_dates, total_contracts 
"""
import Orpheus
import datetime
import logging
import os

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
log_filename = os.path.join(os.getcwd(), 'scriptlogg.log')
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()
    ]
)

logging.info("Logging is set up.")

def SUPERDOWNLOAD_OPTION_OHLC(ticker):
    datacollector = Orpheus.Charts().datacollector
    charts = Orpheus.Charts()
#retrieves tickers unique Id or creates one in options.tickers
    sid = charts.conn.get_or_create_root(ticker)
    logging.info(f"🆔 Working on SID: {sid} for ticker: {ticker}")   

# Collects expirationdates and inserts it to options.expiration
    expiry_list = datacollector.collect_expiration_dates({'root':  ticker})
    charts.conn.insert_expiration_dates(sid,[datetime.datetime.strptime(str(date), "%Y%m%d").date() for date in expiry_list])
    exp_list=[date for date in expiry_list if date>20170101]
    total_expiration_dates = len(exp_list)
    logging.info(f"📅 Found {len(expiry_list)} expiration dates, Only collecting for {total_expiration_dates} after filtering")

# Collects dates available and downloads tick data for option contract.
    current_expiration=1
    for exp_date in exp_list:
        params={'root': ticker, 'exp':exp_date, 'sec': 'option', 'rec':'ohlc'}
        dates_theta = datacollector.collect_theta_list(paramaters=params)
        dates_avail= [date for date in dates_theta if date>20160101]
        params = {'selector': "ohlc", 'exp': exp_date, 'start_date': dates_avail[0], 'end_date': dates_avail[len(dates_avail) -1 ] , 'root': ticker, 'ivl': 60000}
        data_in_expiration = datacollector.theta_bulk_options(params)

# THIS BLOCK IS PER EXPIRATION DATE!!! ther can be 450+ expiration dates per ticker plan accordingly
        if data_in_expiration:
            current_contract = 1
            total_contracts = len(data_in_expiration[0]['response'])
            exp_chain_data=[]
            contracts_in_expiry_tick_list = []
            
    # THIS BLOCK IS PER CONTRACT!! there can be 80+ contracts per expiration date    
            for entry in data_in_expiration[0]['response']:
                contract = (entry['contract'])
                contract['']
                ticks = entry['ticks']
                date_base = datetime.datetime.strptime(str(ticks[-1][-1]), "%Y%m%d")
                contract_id = charts.conn.get_or_create_contract(contract)

        # THIS BLOCK IS PER CONTRACT MINUTE! there can be 400000+ ticks per contract plan accordingly.        
                for tick in ticks:
                    ms_of_day = tick[0]
                    time = date_base + datetime.timedelta(milliseconds=ms_of_day)
                    contracts_in_expiry_tick_list.append((time, contract_id, *tick))
                
                current_contract+=1
        current_expiration+=1
        #charts.conn.insert_option_data(contracts_in_expiry_tick_list, theta=True)
    

SUPERDOWNLOAD_OPTION_OHLC('NVDA')

In [1]:
import httpx  # install via pip install httpx
import Orpheus

conn = Orpheus.Charts().conn

BASE_URL = "http://127.0.0.1:25510/v2"  # all endpoints use this URL base
url = BASE_URL + '/hist/stock/eod'

tick_dickt = {}
tuples = conn.insert_new_command("SELECT * FROM tickers")

for root in [t[0] for t in tuples]:
  try:
    params = {
      'root': root,
      'start_date': '20250422',
      'end_date': '20250429',
    }

    response = httpx.get(url, params=params, timeout=60)  
    res = response.json()
    total = 0
    for r in res['response']:
        total += r[6]
    avg_vol = int(total / len(res['response']))
    print(f"Average volume for {root} is {avg_vol}")
    tick_dickt[root] = avg_vol
  except Exception as e:
    print(f"Error for {root}: {e}")
    continue

# sample_dict = {'a': 5, 'b': 2, 'c': 9, 'd': 1, 'e': 5, 'f': 6}
# sorted_dict = dict(sorted(sample_dict.items(), key=lambda item: item[1]))

print(tick_dickt)
import Orpheus

conn = Orpheus.Charts().conn
conn.copy_csv("options.quotes", '/home/r/QuantLab/externalDisk/AAPL_2024_quotes.csv')

ModuleNotFoundError: No module named 'Orpheus'

In [14]:
import sys
sys.path.append('/home/r/Workspace')  # Not the Orpheus folder, its parent!
from Orpheus.Charts import DataCollector
from Orpheus.Charts import DatabaseConnection
from Orpheus.Charts import config
import pandas as pd

db = DatabaseConnection()
dataC= DataCollector(db, config)
columns =['time', 'open', 'high', 'low', 'close', 'volume']

#spy = dataC.get_stocks_minute('SPY')
spy = db.get_stock_data('SPY', resolution='minute', start_time='2025-01-01')
spy_df = pd.DataFrame(spy, columns=columns)
spy_df.set_index('time', inplace=True)



print(spy_df[20:])

                               open      high       low     close    volume
time                                                                       
2025-01-02 09:40:00+00:00  586.2700  586.8400  586.1400  586.4849    176752
2025-01-02 09:40:00+00:00  245.2300  245.7599  245.0800  245.7500    284116
2025-01-02 09:41:00+00:00  586.5150  586.8600  586.3687  586.5940    111028
2025-01-02 09:41:00+00:00  245.7426  245.7426  245.1600  245.2250    252975
2025-01-02 09:42:00+00:00  245.2500  245.6700  244.8000  245.5550    441448
...                             ...       ...       ...       ...       ...
2025-06-03 15:58:00+00:00  203.3800  203.4300  203.3322  203.3750    279702
2025-06-03 15:58:00+00:00  596.4450  596.4950  596.3200  596.4000    780023
2025-06-03 16:30:00+00:00  201.3500  203.7700  200.9550  203.2700  46381567
2025-06-03 16:30:00+00:00  592.3400  597.0800  591.8500  596.0900  63606204
2025-06-03 20:30:00+00:00  592.3400  597.0800  591.8500  596.0900  63606204

[81410 rows

In [5]:
import sys
print(sys.executable)


/home/r/Workspace/.venv/bin/python
